In [38]:
import numpy as np
from PIL import Image
from pydub import AudioSegment
from pydub.generators import Sine
from io import BytesIO
import requests


In [ ]:

def load_image_from_url(image_url):
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content)).convert('L')  # Convert image to grayscale
        image_data = np.array(image)
        return image_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the image: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


In [ ]:
def load_image_from_file(image_path):
    try:
        image = Image.open(image_path).convert('L')  # Convert image to grayscale
        image_data = np.array(image)
        return image_data
    except Exception as e:
        print(f"Error loading the image: {e}")
        return None


In [ ]:
def map_brightness_to_frequency(image_data, min_freq=220, max_freq=880):
    if image_data is None:
        return None
    norm_image_data = image_data / 255.0  # Normalize to [0, 1]
    frequencies = min_freq + (max_freq - min_freq) * norm_image_data
    return frequencies


In [ ]:
def generate_sound(frequencies, duration=50):
    if frequencies is None:
        return None
    sound = AudioSegment.silent(duration=0)
    for freq in frequencies.flatten():
        tone = Sine(freq).to_audio_segment(duration=duration)
        sound += tone
    return sound


In [ ]:
def image_to_sound(image_source, is_url=True, duration_per_tone=50, min_freq=220, max_freq=880, output_file="output_sound.wav"):
    # Load image
    if is_url:
        image_data = load_image_from_url(image_source)
    else:
        image_data = load_image_from_file(image_source)
    
    if image_data is None:
        print("Failed to load image. Exiting.")
        return
    
    # Map brightness to frequency
    frequencies = map_brightness_to_frequency(image_data, min_freq, max_freq)
    
    # Generate sound
    sound = generate_sound(frequencies, duration_per_tone)
    
    # Export the sound to a file
    sound.export(output_file, format="wav")
    print(f"Sound saved to {output_file}")

# Example usage for URL
image_url = "https://example.com/path/to/your/image.jpg"
image_to_sound(image_url, is_url=True, duration_per_tone=50, min_freq=220, max_freq=880, output_file="output_sound.wav")

# Example usage for local file
# image_path = "your_image_file.jpg"
# image_to_sound(image_path, is_url=False, duration_per_tone=50, min_freq=220, max_freq=880, output_file="output_sound.wav")
